In this challenge, you will create a book recommendation algorithm using K-Nearest Neighbors.

You will use the Book-Crossings dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
should return:

[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301],
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]



Notice that the data returned from get_recommends() is a list. The first element in the list is the book title passed into the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed into the function.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-24 15:09:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   149MB/s    in 0.2s    

2024-03-24 15:09:30 (149 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [ ]:
# clean data for statistical significance
num_ratings= df_ratings.groupby('user').size()
usersinclude  = num_ratings[num_ratings>=200].index # users with less than 200 ratings
num_ratings= df_ratings.groupby('isbn').size()
booksinclude  = num_ratings[num_ratings>=100].index # books with less than 100 ratings (independent of the users)
ratings_new = df_ratings[df_ratings['user'].isin(usersinclude)]
ratings_new = ratings_new[ratings_new['isbn'].isin(booksinclude)]


In [ ]:
# merge df_books and ratings_new
merged_df = pd.merge(df_books, ratings_new, on='isbn', how='inner')
merged_df_clean = merged_df.drop_duplicates(["title", "user"]) # remove all duplicates (otherwise .pivot does not work)

df_pivot = merged_df_clean.pivot(index='title', columns='user', values='rating').fillna(0) # fillna is required


In [ ]:
# Create the KNN model
n_neighbors = 5
nbrs_model = NearestNeighbors(n_neighbors=n_neighbors+1,algorithm='auto',metric='cosine').fit(df_pivot.values) # n_neigbors needs one more to include the book title we want to find additional 5 books; cosine is needed, otherwise output differs from test solution


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances,indices = nbrs_model.kneighbors(df_pivot.loc[book].values.reshape(1,-1)) # get neighbors for the book title, reshape required for function
  distances= distances.flatten().tolist()
  indices= indices.flatten().tolist()

  nearest_books= df_pivot.index[indices[1:]].tolist()
  nearest_books_dist = distances[1:]
  book_list=[]

  for i in range(len(nearest_books)-1,0,-1): # reversed order required since solution wants title with highest distance first
    book_list.append([nearest_books[i],nearest_books_dist[i]])

  recommended_books = [df_pivot.index[indices[0]],book_list]

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074]]]
You passed the challenge! 🎉🎉🎉🎉🎉
